In [3]:
%pylab inline
import numpy as numpy
#import modin.pandas as pd
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import scipy.stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

Populating the interactive namespace from numpy and matplotlib


In [4]:
!ls data

ls: cannot access 'data': No such file or directory


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_train = pd.read_csv('/content/drive/MyDrive/data/train_data.csv', header = None) 
df_test = pd.read_csv('/content/drive/MyDrive/data/test_data.csv', header = None) 
df_train_labels = pd.read_csv('/content/drive/MyDrive/data/train_labels.csv', header = None)

In [7]:
X = df_train
y = df_train_labels

X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=0, stratify = y)

In [8]:
pca = PCA(n_components = 2, whiten = True)

In [9]:
X_train_standarized = StandardScaler().fit_transform(X_train)
X_test_standarized = StandardScaler().fit_transform(X_test)

In [10]:
X_train_pca = pca.fit_transform(X_train_standarized)
X_test_pca = pca.fit_transform(X_test_standarized)

In [11]:
X_train_pca = pd.DataFrame(X_train_pca, columns = ['c1', 'c2'])
X_train_pca = X_train_pca.assign(labels=(y_train).values)
X_train_pca.sample(5)

,c1,c2,labels
1278,-1.128355,-1.858614,1
2373,0.782798,1.291112,1
1132,-1.198048,-0.677178,1
1643,-0.506690,2.103280,1
2480,1.460135,1.355926,-1


In [12]:
kfold = StratifiedKFold(n_splits=10)

In [16]:
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [0.1],
                  'C': [10]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsSVMC.fit(X_train_standarized,y_train)

SVMC_best = gsSVMC.best_estimator_

gsSVMC.best_score_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed: 50.5min finished
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9000719315514502

In [17]:
RFC = RandomForestClassifier()

rf_param_grid = {"max_depth": [None],
              "max_features": [10],
              "min_samples_split": [2],
              "min_samples_leaf": [10],
              "bootstrap": [False],
              "n_estimators" :[10],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train_standarized,y_train)

RFC_best = gsRFC.best_estimator_

gsRFC.best_score_



Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   39.1s finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


0.9000719315514502

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
   
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

g = plot_learning_curve(gsSVMC.best_estimator_,"SVC learning curves",X_train,y_train,cv=kfold)
g = plot_learning_curve(gsRFC.best_estimator_,"RF mearning curves",X_train,y_train,cv=kfold)

In [ ]:
test_Survived_SVMC = pd.Series(SVMC_best.predict(test), name="SVC")
test_Survived_RFC = pd.Series(RFC_best.predict(test), name="RFC")

ensemble_results = pd.concat([test_Survived_RFC,test_Survived_ExtC,test_Survived_AdaC,test_Survived_GBC, test_Survived_SVMC],axis=1)

g= sns.heatmap(ensemble_results.corr(),annot=True)

In [19]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best),('svc', SVMC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train_standarized, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
test_label = pd.Series(votingC.predict(df_test), name="Predict")
results = pd.concat([df_train_labels, test_label],axis=1)

In [23]:
results.to_csv('results_labels.csv',index=False)